<a href="https://colab.research.google.com/github/Kalit31/Data-Mining-Assignment/blob/main/SJ-LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#Import libraries
import pandas as pd
import numpy as np
import cv2
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback, TensorBoard
from keras.layers import Input, Embedding, Bidirectional, LSTM, Conv1D, MaxPooling1D, BatchNormalization, Flatten, Dense, Dropout, Reshape, Concatenate, Masking
from keras.regularizers import l2
from keras.utils import Sequence, to_categorical
from keras.optimizers import Adam
from keras.backend import epsilon
from keras.models import model_from_json
from keras.models import Model, Sequential

import nltk
from nltk.tokenize import RegexpTokenizer

from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, classification_report,accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

**Import dataset** 

In [3]:
trainFilePath = 'dataset/train2.tsv'
testFilePath = 'dataset/test2.tsv'
validationFilePath = 'dataset/val2.tsv'

PATH = "/content/drive/My Drive/"

In [4]:
df_train = pd.read_csv(PATH+trainFilePath, delimiter='\t',  names=["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State", "Party",
                         "Barely True Cnt", "False Cnt", "Half True Cnt", "Mostly True Cnt", "Pants on Fire Cnt", "Context", "Justification"])

df_test = pd.read_csv(PATH+testFilePath, delimiter='\t',  names=["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State", "Party",
                         "Barely True Cnt", "False Cnt", "Half True Cnt", "Mostly True Cnt", "Pants on Fire Cnt", "Context", "Justification"])


df_validation = pd.read_csv(PATH+validationFilePath, delimiter='\t', names=["ID", "Label", "Statement", "Subject", "Speaker", "Job Title", "State", "Party",
                         "Barely True Cnt", "False Cnt", "Half True Cnt", "Mostly True Cnt", "Pants on Fire Cnt", "Context", "Justification"])

In [5]:
df_train.head()

,ID,Label,Statement,Subject,Speaker,Job Title,State,Party,Barely True Cnt,False Cnt,Half True Cnt,Mostly True Cnt,Pants on Fire Cnt,Context,Justification
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."


**Data Preprocessing**

In [6]:
def dataCleaning(df,field):
    df[field] = df[field].str.replace(r"@\S+", "")
    df[field] = df[field].str.replace(r"[^A-Za-z0-9]", " ")
    df[field] = df[field].str.replace(r"(),!?@\'\`\"\_\n", " ")
    df[field] = df[field].str.replace(r"@", "at")
    df[field] = df[field].str.replace(r"http\S+", "")
    df[field] = df[field].str.replace(r"http", "")
    df[field] = df[field].str.lower()
    return df

def dataPreprocessing(df):
    df = df[df['ID'].notna()]
    df = df[df['Barely True Cnt'].notna()]
    df = df[df['False Cnt'].notna()]
    df = df[df['Mostly True Cnt'].notna()]
    df = df[df['Pants on Fire Cnt'].notna()]
    df = df[df['Half True Cnt'].notna()]

    df['ID'] = df['ID'].str.split(".", n = 1, expand = True) 
    
    df = dataCleaning(df,'Statement')
    df = dataCleaning(df,'Subject')
    df = dataCleaning(df,'Speaker')
    df = dataCleaning(df,'Job Title')
    df = dataCleaning(df,'State')
    df = dataCleaning(df,'Party')
    df = dataCleaning(df,'Context')
    df = dataCleaning(df,'Justification')    
    
    le_multi = LabelEncoder()
    df.loc[:, 'Multi Class Label'] = le_multi.fit_transform(df.Label)
    print("Label assignments: " + str({l: i for i, l in enumerate(le_multi.classes_)}))   
    
    df['Binary Label'] = df.Label.apply(lambda x: 1 if x in ['false','pants-fire','barely-true']  else 0)

    df = df.fillna('None')

    return df

In [7]:
df_train = dataPreprocessing(df_train)
df_validation = dataPreprocessing(df_validation)
df_test = dataPreprocessing(df_test)

Label assignments: {'barely-true': 0, 'false': 1, 'half-true': 2, 'mostly-true': 3, 'pants-fire': 4, 'true': 5}
Label assignments: {'barely-true': 0, 'false': 1, 'half-true': 2, 'mostly-true': 3, 'pants-fire': 4, 'true': 5}
Label assignments: {'barely-true': 0, 'false': 1, 'half-true': 2, 'mostly-true': 3, 'pants-fire': 4, 'true': 5}


**Embed data fields**

In [9]:
t = Tokenizer()
  
def get_embeddings(data, max_length):
  t.fit_on_texts(data)
  encoded_docs = t.texts_to_sequences(data)
  padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
  return padded_docs

In [10]:
def embed_fields(df):
  return [np.asarray(get_embeddings(df['Statement'], 50)).astype(np.int32),np.asarray(get_embeddings(df['Subject'], 10)).astype(np.int32),np.asarray(get_embeddings(df['Context'], 25)).astype(np.int32),
          np.asarray(get_embeddings(df['Speaker'], 5)).astype(np.int32),np.asarray(get_embeddings(df['Party'], 5)).astype(np.int32),np.asarray(get_embeddings(df['Job Title'], 20)).astype(np.int32),
          np.asarray(get_embeddings(df['State'], 5)).astype(np.int32),np.asarray(get_embeddings(df['Justification'], 150)).astype(np.int32),embed_metadata(df)]

In [11]:
train_stmt_x = get_embeddings(df_train['Statement'], 50)
train_just_x = get_embeddings(df_train['Justification'], 150)

val_stmt_x = get_embeddings(df_validation['Statement'], 50)
val_just_x = get_embeddings(df_validation['Justification'], 150)

test_stmt_x = get_embeddings(df_test['Statement'], 50)
test_just_x = get_embeddings(df_test['Justification'], 150)

In [12]:
vocab_size = len(t.word_index) + 1
print("VOCABULARY SIZE: "+str(vocab_size))

VOCABULARY SIZE: 27387


In [13]:
embeddings_index = dict()
f = open(PATH + 'dataset/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [14]:
def create_embedding_matrix():
  embedding_matrix = np.zeros((vocab_size, 100))
  for word, i in t.word_index.items():
	  embedding_vector = embeddings_index.get(word)
	  if embedding_vector is not None:
		  embedding_matrix[i] = embedding_vector
  return embedding_matrix

In [15]:
embedding_matrix = create_embedding_matrix()

In [16]:
def get_features(input_length):
  input_tensor = Input((input_length,))
  X = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=input_length, trainable=False)(input_tensor)
  X = Bidirectional(LSTM(32, return_sequences=True))(X)
  X = Bidirectional(LSTM(16, return_sequences=True))(X)
  X = Flatten()(X)
  X = Dense(1024, activation='relu', kernel_regularizer=l2(0.0))(X)
  X = Dropout(0.3)(X)
  X = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(X)
  X = Reshape((8,16))(X)
  X = Conv1D(128,3, padding='same', activation='relu', kernel_regularizer=l2(0.0))(X)
  X = MaxPooling1D(2)(X)
  X = BatchNormalization()(X)
  X = Flatten()(X)
  output_tensor = Dense(128, activation='relu', kernel_regularizer=l2(0.0))(X)
  output_tensor = Dropout(0.3)(output_tensor)
  return input_tensor, output_tensor

**Binary Classification Model**

In [17]:
stmt_input, stmt_ftrs = get_features(train_stmt_x.shape[1])
just_input, just_ftrs = get_features(train_just_x.shape[1])

out_1 = Concatenate()([stmt_ftrs, just_ftrs])
out = Dense(1028, activation='relu', kernel_regularizer=l2(0.0))(out_1)
out = Dropout(0.3)(out)
out = Dense(256, activation='relu', kernel_regularizer=l2(0.0))(out)
out = Dropout(0.3)(out)
out = Dense(2, activation='sigmoid')(out)
model_binary = Model(inputs = [stmt_input, just_input], outputs = out)
model_binary.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 100)      2738700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 100)     2738700     input_2[0][0]                    
______________________________________________________________________________________________

In [18]:
train_y = to_categorical(df_train['Binary Label'], 2)
val_y = to_categorical(df_validation['Binary Label'], 2)
test_y = to_categorical(df_test['Binary Label'], 2)

In [19]:
stop = EarlyStopping(monitor="val_acc", patience=20, mode="max")
reduce_lr = ReduceLROnPlateau(monitor="val_acc", factor=0.1, patience=10, min_lr=1e-6, verbose=1, mode="max")
optimizer = Adam(lr = 0.001)
model_binary.compile(optimizer, loss='binary_crossentropy', metrics=['acc'])

In [20]:
model_binary.fit([train_stmt_x, train_just_x], train_y,
          epochs=10, verbose=1, validation_data=([val_stmt_x, val_just_x], val_y), callbacks=[reduce_lr, stop])

Epoch 1/10
320/320 [==============================] - 57s 47ms/step - loss: 2.0670 - acc: 0.5526 - val_loss: 0.7050 - val_acc: 0.5389
Epoch 2/10
320/320 [==============================] - 12s 38ms/step - loss: 0.6957 - acc: 0.5625 - val_loss: 0.7189 - val_acc: 0.5210
Epoch 3/10
320/320 [==============================] - 12s 37ms/step - loss: 0.6880 - acc: 0.5699 - val_loss: 0.7059 - val_acc: 0.5055
Epoch 4/10
320/320 [==============================] - 12s 37ms/step - loss: 0.6750 - acc: 0.5953 - val_loss: 0.7027 - val_acc: 0.5171
Epoch 5/10
320/320 [==============================] - 12s 37ms/step - loss: 0.6751 - acc: 0.5938 - val_loss: 0.7225 - val_acc: 0.4860
Epoch 6/10
320/320 [==============================] - 12s 38ms/step - loss: 0.6705 - acc: 0.6040 - val_loss: 0.7344 - val_acc: 0.5156
Epoch 7/10
320/320 [==============================] - 12s 38ms/step - loss: 0.6513 - acc: 0.6322 - val_loss: 0.7458 - val_acc: 0.5047
Epoch 8/10
320/320 [==============================] - 12s 37ms

In [21]:
val_loss, val_acc = model_binary.evaluate([val_stmt_x, val_just_x], val_y,
                                   verbose=1)
print('Validation Accuracy: %f' % (val_acc))
test_loss, test_acc = model_binary.evaluate([test_stmt_x, test_just_x],
                                     test_y, verbose=1)
print('Test Accuracy: %f' % (test_acc))

41/41 [==============================] - 1s 17ms/step - loss: 0.7534 - acc: 0.4759
Validation Accuracy: 0.475857
40/40 [==============================] - 1s 17ms/step - loss: 0.7468 - acc: 0.4925
Test Accuracy: 0.492502


**Multi class classification model**

In [23]:
train_y = to_categorical(df_train['Multi Class Label'], 6)
val_y = to_categorical(df_validation['Multi Class Label'], 6)
test_y = to_categorical(df_test['Multi Class Label'], 6)

In [24]:
stmt_input, stmt_ftrs = get_features(train_stmt_x.shape[1])
just_input, just_ftrs = get_features(train_just_x.shape[1])

out_1 = Concatenate()([stmt_ftrs, just_ftrs])
out = Dense(1028, activation='relu', kernel_regularizer=l2(0.0))(out_1)
out = Dropout(0.3)(out)
out = Dense(256, activation='relu', kernel_regularizer=l2(0.0))(out)
out = Dropout(0.3)(out)
out = Dense(6, activation='softmax')(out)
model_multi = Model(inputs = [stmt_input, just_input], outputs = out)
model_multi.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 50, 100)      2738700     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 150, 100)     2738700     input_6[0][0]                    
____________________________________________________________________________________________

In [25]:
stop = EarlyStopping(monitor="val_acc", patience=20, mode="max")
reduce_lr = ReduceLROnPlateau(monitor="val_acc", factor=0.1, patience=10, min_lr=1e-6, verbose=1, mode="max")
optimizer = Adam(lr = 0.001)
model_multi.compile(optimizer, loss='binary_crossentropy', metrics=['acc'])

In [28]:
model_multi.fit([train_stmt_x, train_just_x], train_y,
          epochs=10, verbose=1, validation_data=([val_stmt_x, val_just_x], val_y), callbacks=[reduce_lr, stop])

Epoch 1/10
320/320 [==============================] - 24s 44ms/step - loss: 1.6353 - acc: 0.2015 - val_loss: 0.4949 - val_acc: 0.1955
Epoch 2/10
320/320 [==============================] - 12s 37ms/step - loss: 0.4565 - acc: 0.2069 - val_loss: 0.4599 - val_acc: 0.2048
Epoch 3/10
320/320 [==============================] - 12s 37ms/step - loss: 0.4526 - acc: 0.2076 - val_loss: 0.4515 - val_acc: 0.1963
Epoch 4/10
320/320 [==============================] - 12s 37ms/step - loss: 0.4468 - acc: 0.2139 - val_loss: 0.4484 - val_acc: 0.1955
Epoch 5/10
320/320 [==============================] - 12s 37ms/step - loss: 0.4448 - acc: 0.2327 - val_loss: 0.4522 - val_acc: 0.1963
Epoch 6/10
320/320 [==============================] - 12s 37ms/step - loss: 0.4426 - acc: 0.2360 - val_loss: 0.4519 - val_acc: 0.1994
Epoch 7/10
320/320 [==============================] - 12s 37ms/step - loss: 0.4411 - acc: 0.2476 - val_loss: 0.4590 - val_acc: 0.1939
Epoch 8/10
320/320 [==============================] - 12s 37ms

In [29]:
val_loss, val_acc = model_multi.evaluate([val_stmt_x, val_just_x], val_y,
                                   verbose=1)
print('Validation Accuracy: %f' % (val_acc))
test_loss, test_acc = model_multi.evaluate([test_stmt_x, test_just_x],
                                     test_y, verbose=1)
print('Test Accuracy: %f' % (test_acc))

41/41 [==============================] - 1s 18ms/step - loss: 0.4655 - acc: 0.1861
Validation Accuracy: 0.186137
40/40 [==============================] - 1s 18ms/step - loss: 0.4650 - acc: 0.1831
Test Accuracy: 0.183110
